In [1]:
!pip install --upgrade transformers
!pip install simpletransformers
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
!pip install pyspellchecker

Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (3.0.2)


In [2]:
import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 12.7 GB  |     Proc size: 109.4 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total     15079MB


In [3]:
import numpy as np
import pandas as pd
from google.colab import files
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')
import gc
from scipy.special import softmax
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
import sklearn
from sklearn.metrics import log_loss
from sklearn.metrics import *
from sklearn.model_selection import *
from ast import literal_eval
import re
from spellchecker import SpellChecker
import random
from sklearn.preprocessing import LabelEncoder
import torch
pd.options.display.max_colwidth = 200

def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

seed_all(2)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [4]:
# spell = SpellChecker()
# def get_correct_words(x):
#     new_sentence = []
#     for i in x.split():
#         check_mispelled = spell.unknown([i])
#         if len(check_mispelled) > 0:
#             for word in check_mispelled:
#                 word_correct = spell.correction(word)
#                 new_sentence.append(word_correct)
#         else:
#             new_sentence.append(i)
#     return ' '.join(new_sentence)

In [5]:
# from albumentations.core.transforms_interface import DualTransform, BasicTransform
# from nltk import sent_tokenize
# class NLPTransform(BasicTransform):
#     """ Transform for nlp task."""
#     LANGS = {
#         'en': 'english'
#     }

#     @property
#     def targets(self):
#         return {"data": self.apply}
    
#     def update_params(self, params, **kwargs):
#         if hasattr(self, "interpolation"):
#             params["interpolation"] = self.interpolation
#         if hasattr(self, "fill_value"):
#             params["fill_value"] = self.fill_value
#         return params

#     def get_sentences(self, text, lang='en'):
#         return sent_tokenize(text, self.LANGS.get(lang, 'english'))

In [6]:
# class SwapWordsTransform(NLPTransform):
#     """ Swap words next to each other """
#     def __init__(self, swap_distance=1, swap_probability=0.1, always_apply=False, p=0.5):
#         """  
#         swap_distance - distance for swapping words
#         swap_probability - probability of swapping for one word
#         """
#         super(SwapWordsTransform, self).__init__(always_apply, p)
#         self.swap_distance = swap_distance
#         self.swap_probability = swap_probability
#         self.swap_range_list = list(range(1, swap_distance+1))

#     def apply(self, data, **params):
#         text, lang = data
#         words = text.split()
#         words_count = len(words)
#         if words_count <= 1:
#             return text, lang

#         new_words = {}
#         for i in range(words_count):
#             if random.random() > self.swap_probability:
#                 new_words[i] = words[i]
#                 continue
    
#             if i < self.swap_distance:
#                 new_words[i] = words[i]
#                 continue
    
#             swap_idx = i - random.choice(self.swap_range_list)
#             new_words[i] = new_words[swap_idx]
#             new_words[swap_idx] = words[i]

#         return ' '.join([v for k, v in sorted(new_words.items(), key=lambda x: x[0])]), lang

In [7]:
# class CutOutWordsTransform(NLPTransform):
#     """ Remove random words """
#     def __init__(self, cutout_probability=0.05, always_apply=False, p=0.5):
#         super(CutOutWordsTransform, self).__init__(always_apply, p)
#         self.cutout_probability = cutout_probability

#     def apply(self, data, **params):
#         text, lang = data
#         words = text.split()
#         words_count = len(words)
#         if words_count <= 1:
#             return text, lang
        
#         new_words = []
#         for i in range(words_count):
#             if random.random() < self.cutout_probability:
#                 continue
#             new_words.append(words[i])

#         if len(new_words) == 0:
#             return words[random.randint(0, words_count-1)], lang

#         return ' '.join(new_words), lang

In [8]:
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/JanathaHack/Janatahack: Independence Day 2020 ML Hackathon/train.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/JanathaHack/Janatahack: Independence Day 2020 ML Hackathon/test.csv')
sample_sub = pd.read_csv('/content/drive/My Drive/Colab Notebooks/JanathaHack/Janatahack: Independence Day 2020 ML Hackathon/sample_submission_UVKGLZE.csv')
train.ABSTRACT = train.ABSTRACT.str.replace("\n"," ")
test.ABSTRACT = test.ABSTRACT.str.replace("\n"," ")

train["text"] = train["TITLE"] + train["ABSTRACT"]
test["text"] = test["TITLE"] + test["ABSTRACT"]

target_labels = ["Computer Science" ,"Physics" , "Mathematics" ,"Statistics" , "Quantitative Biology" , "Quantitative Finance"]
train['label'] = train[target_labels].values.tolist()

le = LabelEncoder()
train['label'] = le.fit_transform(train['label'].astype(str))
train = train[["text","label"]]
test = test[["text"]]
test["label"] = 0

In [9]:
# train['text']= train['text'].apply(lambda x: re.sub(r"[^A-Za-z]", " ", x))
# test['text'] = test['text'].apply(lambda x: re.sub(r"[^A-Za-z]", " ", x))
#--------------------------------------------------------
# train['text'] = train['text'].apply(lambda x:get_correct_words(x))
# test['text'] = test['text'].apply(lambda x:get_correct_words(x))

In [10]:
# train.to_csv("/content/drive/My Drive/Colab Notebooks/JanathaHack/Janatahack: Independence Day 2020 ML Hackathon/regex_train.csv",index=False)
# test.to_csv("/content/drive/My Drive/Colab Notebooks/JanathaHack/Janatahack: Independence Day 2020 ML Hackathon/regex_test.csv",index=False)

In [11]:
# transform = SwapWordsTransform(p=1.0, swap_distance=1, swap_probability=0.2)
# tr=CutOutWordsTransform(p=1.0, cutout_probability=0.2)

# lang = 'en'

# for i in range(len(train)):
#         text = train['text'][i]
#         text=transform(data=(text, lang))['data'][0]
#         train['text'][i]=tr(data=(text, lang))['data'][0]

In [12]:
# print(train['text'].apply(lambda x: len(x.split())).describe())

In [13]:
train

,text,label
0,"Reconstructing Subject-Specific Effect Maps Predictive models allow subject-specific inference when analyzing disease related alterations in neuroimaging data. Given a subject's data, inference c...",13
1,"Rotation Invariance Neural Network Rotation invariance and translation invariance have great values in image recognition tasks. In this paper, we bring a new architecture in convolutional neural ...",13
2,"Spherical polyharmonics and Poisson kernels for polyharmonic functions We introduce and develop the notion of spherical polyharmonics, which are a natural generalisation of spherical harmonics. I...",6
3,A finite element approximation for the stochastic Maxwell--Landau--Lifshitz--Gilbert system The stochastic Landau--Lifshitz--Gilbert (LLG) equation coupled with the Maxwell equations (the so call...,6
4,Comparative study of Discrete Wavelet Transforms and Wavelet Tensor Train decomposition to feature extraction of FTIR data of medicinal plants Fourier-transform infra-red (FTIR) spectra of sample...,16
...,...,...
20967,Contemporary machine learning: a guide for practitioners in the physical sciences Machine learning is finding increasingly broad application in the physical sciences. This most often involves bui...,21
20968,Uniform diamond coatings on WC-Co hard alloy cutting inserts deposited by a microwave plasma CVD Polycrystalline diamond coatings have been grown on cemented carbide substrates with different asp...,9
20969,"Analysing Soccer Games with Clustering and Conceptors We present a new approach for identifying situations and behaviours, which we call ""moves"", from soccer games in the 2D simulation league. Be...",13
20970,On the Efficient Simulation of the Left-Tail of the Sum of Correlated Log-normal Variates The sum of Log-normal variates is encountered in many challenging applications such as in performance ana...,7


In [14]:
train

,text,label
0,"Reconstructing Subject-Specific Effect Maps Predictive models allow subject-specific inference when analyzing disease related alterations in neuroimaging data. Given a subject's data, inference c...",13
1,"Rotation Invariance Neural Network Rotation invariance and translation invariance have great values in image recognition tasks. In this paper, we bring a new architecture in convolutional neural ...",13
2,"Spherical polyharmonics and Poisson kernels for polyharmonic functions We introduce and develop the notion of spherical polyharmonics, which are a natural generalisation of spherical harmonics. I...",6
3,A finite element approximation for the stochastic Maxwell--Landau--Lifshitz--Gilbert system The stochastic Landau--Lifshitz--Gilbert (LLG) equation coupled with the Maxwell equations (the so call...,6
4,Comparative study of Discrete Wavelet Transforms and Wavelet Tensor Train decomposition to feature extraction of FTIR data of medicinal plants Fourier-transform infra-red (FTIR) spectra of sample...,16
...,...,...
20967,Contemporary machine learning: a guide for practitioners in the physical sciences Machine learning is finding increasingly broad application in the physical sciences. This most often involves bui...,21
20968,Uniform diamond coatings on WC-Co hard alloy cutting inserts deposited by a microwave plasma CVD Polycrystalline diamond coatings have been grown on cemented carbide substrates with different asp...,9
20969,"Analysing Soccer Games with Clustering and Conceptors We present a new approach for identifying situations and behaviours, which we call ""moves"", from soccer games in the 2D simulation league. Be...",13
20970,On the Efficient Simulation of the Left-Tail of the Sum of Correlated Log-normal Variates The sum of Log-normal variates is encountered in many challenging applications such as in performance ana...,7


In [15]:
wordcount = train['text'].str.count(' ') + 1
wordcount.mean()

159.91989319092121

In [16]:
test

,text,label
0,"Closed-form Marginal Likelihood in Gamma-Poisson Matrix Factorization We present novel understandings of the Gamma-Poisson (GaP) model, a probabilistic matrix factorization model for count data. ...",0
1,Laboratory mid-IR spectra of equilibrated and igneous meteorites. Searching for observables of planetesimal debris Meteorites contain minerals from Solar System asteroids with different propertie...,0
2,Case For Static AMSDU Aggregation in WLANs Frame aggregation is a mechanism by which multiple frames are combined into a single transmission unit over the air. Frames aggregated at the AMSDU leve...,0
3,"The $Gaia$-ESO Survey: the inner disk intermediate-age open cluster NGC 6802 Milky Way open clusters are very diverse in terms of age, chemical composition, and kinematic properties. Intermediate...",0
4,Witness-Functions versus Interpretation-Functions for Secrecy in Cryptographic Protocols: What to Choose? Proving that a cryptographic protocol is correct for secrecy is a hard task. One of the s...,0
...,...,...
8984,Supporting mixed-datatype matrix multiplication within the BLIS framework We approach the problem of implementing mixed-datatype support within the general matrix multiplication (GEMM) operation ...,0
8985,"An axiomatic basis for Blackwell optimality In the theory of Markov decision processes (MDPs), a Blackwell optimal policy is a policy that is optimal for every discount factor sufficiently close ...",0
8986,GeneVis - An interactive visualization tool for combining cross-discipline datasets within genetics GeneVis is a web-based tool to visualize complementary data sets of different disciplines withi...,0
8987,Quantifying the causal effect of speed cameras on road traffic accidents via an approximate Bayesian doubly robust estimator This paper quantifies the effect of speed cameras on road traffic coll...,0


In [17]:
train['label'].nunique()

24

In [31]:
model = ClassificationModel('roberta', 'roberta-large', use_cuda=True,num_labels=24, args={'train_batch_size':16,
                                                                         'reprocess_input_data': True,
                                                                         'overwrite_output_dir': True,
                                                                         'fp16': False,
                                                                         'do_lower_case': False,
                                                                         'num_train_epochs': 2,
                                                                         'max_seq_length': 16,
                                                                         'regression': False,
                                                                         'manual_seed': 2,
                                                                         "learning_rate":5e-5,
                                                                         'weight_decay':0.0,
                                                                         "save_eval_checkpoints": False,
                                                                         "save_model_every_epoch": False,
                                                                         "silent": False})

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.o

In [ ]:
model.train_model(train)

In [ ]:
tst_result, tst_model_outputs, tst_wrong_predictions = model.eval_model(test)

In [21]:
preds = softmax(tst_model_outputs,axis=1)

In [22]:
import pickle
with open("/content/drive/My Drive/Colab Notebooks/JanathaHack/Janatahack: Independence Day 2020 ML Hackathon/preds3.pkl", 'wb') as f:
    pickle.dump(preds, f)



# with open("/content/drive/My Drive/Colab Notebooks/JanathaHack/Janatahack: Independence Day 2020 ML Hackathon/preds1.pkl", 'rb') as f:
#     preds1 = pickle.load(f)
# with open("/content/drive/My Drive/Colab Notebooks/JanathaHack/Janatahack: Independence Day 2020 ML Hackathon/preds2.pkl", 'rb') as f:
#     preds2 = pickle.load(f)

In [23]:
preds_fin = preds1*0.8 + preds2*0.2

NameError: ignored

In [24]:
# finalpreds = [np.argmax(x) for x in preds_fin]
finalpreds = [np.argmax(x) for x in preds]

In [25]:
sample_sub["target"] = finalpreds
sample_sub["target"] = le.inverse_transform(sample_sub["target"])
sample_sub.loc[:,'target'] = sample_sub.loc[:,'target'].apply(lambda x: literal_eval(x))
sample_sub[target_labels] = pd.DataFrame(sample_sub.target.tolist(), index= sample_sub.index)
sample_sub.drop("target",axis=1,inplace = True)

In [26]:
sample_sub.to_csv("indpday_submit8.csv",index = False)

In [27]:
sample_sub["target"] = finalpreds

In [28]:
sample_sub[sample_sub["target"]==19]

,ID,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance,target


In [29]:
sample_sub["target"].value_counts()

9    8989
Name: target, dtype: int64

In [ ]:
train["label"].value_counts()

In [30]:
test

,text,label
0,"Closed-form Marginal Likelihood in Gamma-Poisson Matrix Factorization We present novel understandings of the Gamma-Poisson (GaP) model, a probabilistic matrix factorization model for count data. ...",0
1,Laboratory mid-IR spectra of equilibrated and igneous meteorites. Searching for observables of planetesimal debris Meteorites contain minerals from Solar System asteroids with different propertie...,0
2,Case For Static AMSDU Aggregation in WLANs Frame aggregation is a mechanism by which multiple frames are combined into a single transmission unit over the air. Frames aggregated at the AMSDU leve...,0
3,"The $Gaia$-ESO Survey: the inner disk intermediate-age open cluster NGC 6802 Milky Way open clusters are very diverse in terms of age, chemical composition, and kinematic properties. Intermediate...",0
4,Witness-Functions versus Interpretation-Functions for Secrecy in Cryptographic Protocols: What to Choose? Proving that a cryptographic protocol is correct for secrecy is a hard task. One of the s...,0
...,...,...
8984,Supporting mixed-datatype matrix multiplication within the BLIS framework We approach the problem of implementing mixed-datatype support within the general matrix multiplication (GEMM) operation ...,0
8985,"An axiomatic basis for Blackwell optimality In the theory of Markov decision processes (MDPs), a Blackwell optimal policy is a policy that is optimal for every discount factor sufficiently close ...",0
8986,GeneVis - An interactive visualization tool for combining cross-discipline datasets within genetics GeneVis is a web-based tool to visualize complementary data sets of different disciplines withi...,0
8987,Quantifying the causal effect of speed cameras on road traffic accidents via an approximate Bayesian doubly robust estimator This paper quantifies the effect of speed cameras on road traffic coll...,0
